### In this notebook we process data extracted from the Philipis ICCA system in Bristol, UK for use in an audit and machine learning project with CORMSIS, Southampton.  

### Data are processed to faciliate rapid analysis by CORMSIS student KQ. De-identification follows standards required by CAG guidelines for clinical audit involving a third party.

Note: The following bash commands respectively remove trailing and preceding lines from the report files. (This is required for clean loading with pandas.read_csv).
* sed -i '$ d' ptassess_physiological_data.rpt
* sed -i '1d' ptassess_physiological_data.rpt 

In [2]:
import numpy as np
import pandas as pd

### 1. First we produce the (patient) summary data for all ICU stays in the dataset:

In [3]:
stays = pd.read_csv('patient_stays_summary.rpt', delimiter='\t')
demographics = pd.read_csv('patient_demographics_heights_weights.rpt', delimiter='\t')

In [4]:
demographics.groupby('interventionId').agg({'longLabel':['count', 'first']}); ## output supressed for data protection

#### Multiple interventionId numbers in use. We expect there to be duplication. For example, "Weight (Admission)" and "Weight (admission)" are frequently both entered for the same encounter. Not clear why.

####  We define an order of priority and select one weight and one height for each ICU stay as follows:

In [5]:
weight_priority = [4615, 2110, 8917, 10098, 2079, 8906, 4853]
height_priority = [2714, 2377]

In [6]:
weights = dict()
heights = dict()

for ii in demographics['encounterId'].unique():
    
    subset = demographics[demographics['encounterId']==ii]
    weight = (np.nan, np.nan)
    height = (np.nan, np.nan)
    
    for wid in weight_priority:
        w = subset[subset['interventionId']==wid]
        if len(w)>0:
            weight = (w['chartTime'].values[0], w['valueNumber'].values[0])
            break
            
    for hid in height_priority:
        h = subset[subset['interventionId']==hid]
        if len(h)>0:
            height = (h['chartTime'].values[0], h['valueNumber'].values[0])
            break
            
    weights[ii] = weight
    heights[ii] = height

#### We now convert the inTime column to type datetime, and then remove MOST sensitive data (inTime will be removed later after being used to convert other time stamps in the data). 

In [7]:
##print stays.dtypes
stays['inTime'] = pd.to_datetime(stays['inTIME'], infer_datetime_format=True)
anon = stays.drop(labels=['tNumber', 'numberOfRecords', 'inTIME', 'outTime'], axis=1)

#### Add in weights and heights as calculated above:

In [8]:
anon['weight'] = anon.encounterId.apply(lambda x: weights[x][1] if x in weights.keys() else np.nan)
anon['height'] = anon.encounterId.apply(lambda x: heights[x][1] if x in heights.keys() else np.nan)

#### Josh Inoue (from Univeristy Hospitals Bristol) has compiled a list of bad encounterId numbers. These are ICU stays which were created in error or involve some other type of data corruption. We load the lists of these ID numbers and then remove them from our  summary data:

In [9]:
erroneous_entries = pd.read_excel('../../working with ICNARC/Philips encounterId Issue List (New).xlsx', sheet_name='encounterId')
erroneous_entries2 = pd.read_excel('../../working with ICNARC/Philips encounterId Issue List (New).xlsx', sheet_name='WW')

In [10]:
err = [e for e in erroneous_entries['encounterId_CIS']]
err = err + [e for e in erroneous_entries['encounterId_Adjusted']]
err = err + [e for e in erroneous_entries2['Corrected encID']]
err = err + [e for e in erroneous_entries2['CIS Patient ID']]
to_drop = [i for i,row in anon.iterrows() if row['encounterId'] in err]
print "%d ICU stays to remove." %len(to_drop)

248 ICU stays to remove.


In [11]:
anon = anon.drop(to_drop, axis=0)

#### We now further reduce the cohort of patients by selecting only those ICU stays which appear in the ICNARC (national audit) database, or those that were admitted during 2019 (for which we do not currently have the ICNARC data). 

Linking to ICNARC ensures that we are looking only at genuine stays, not test patients or erroneous entries etc.

Note: the patients for which we have ICNARC data we have outcome information (in-hospital mortality), diagnosis and basic medical history.

In [12]:
icnarc = pd.read_csv('../../working with ICNARC/ICNARC 2015-2018 encounterIds and Readmissions.TXT')
to_drop = [i for i,row in anon.iterrows() if row['encounterId'] not in icnarc['CIS Patient ID'].values]
print "%d patients not in this extract of the ICANRC data." %len(to_drop)

807 patients not in this extract of the ICANRC data.


In [13]:
without_2019 = anon.drop(to_drop, axis=0)
only_2019 = anon[anon['inTime']>='2019']
anon = without_2019.append(only_2019)

In [14]:
print "There are now %d unique ICU stays in the data." %len(anon.encounterId.unique())

There are now 5413 unique ICU stays in the data.


#### We now add in the month of admission and day of admission, both as incremental integers beginning from the earliest admission date :

In [15]:
# Previously used first inTime, but need to start from midnight ...
# DAY_0 = anon['inTime'].iloc[0]
DAY_0 = pd.Timestamp(year=2015, month=2, day=1, hour=0, minute=0, second=0)  

In [16]:
anon['admission month'] = anon['inTime'].apply(lambda x : (x.year - 2015) * 12 + x.month)
anon['admission day'] = anon['inTime'].apply(lambda x : (x - DAY_0).days)
anon['admission time'] = anon['inTime'].apply(lambda x: x.time())

### 2. Next we prepare the fluid balance targets.

In [17]:
targets = pd.read_csv('fluid_balance_target.rpt', delimiter='\t')

#### The interventionId 20324 contains gibberish (corrupt data?), therefore we have fewer patients than expected using only 52111:

In [18]:
targets = targets[targets['interventionId']==52111]

#### We convert chartTime to datetime format and then calculate 'minutes since admission' and 'record day' by joining to the patient summary data (anon). We also get just time of day from 'time': 

In [19]:
targets['time'] = pd.to_datetime(targets['chartTime'], infer_datetime_format=True)
targets = targets.merge(anon, on='encounterId')
targets['minutes since admission'] = (targets['time'] - targets['inTime']) / pd.Timedelta(minutes=1)
targets['record day'] = targets['time'].apply(lambda x:  (x- DAY_0).days)
targets['time of day'] = targets['time'].apply(lambda x: x.time())

#### Select only relevant columns and save:

In [20]:
targets = targets[['minutes since admission', 'encounterId', 'valueString', 'record day', 'time of day']]
targets.to_csv('anonymised_targets.csv')

### 3. Then process 'Total balances' (fluids). 

In [21]:
totals = pd.read_csv('total_balances.rpt', delimiter='\t')
totals = totals[['interventionId', 'encounterId', 'chartTime', 'careProviderId', 'hourTotal', 'cumTotal', 'unitOfMeasure', 'longLabel']]

#### We drop any encounterId numbers that do not appear in our patient summary data frame (anon):

In [22]:
to_drop = [i for i,row in totals.iterrows() if row['encounterId'] not in anon['encounterId'].values]
totals = totals.drop(to_drop, axis=0)

In [23]:
totals['time'] = pd.to_datetime(totals['chartTime'], infer_datetime_format=True)
totals = totals.merge(anon, on='encounterId')
totals['minutes since admission'] = (totals['time'] - totals['inTime']) / pd.Timedelta(minutes=1)
totals['record day'] = totals['time'].apply(lambda x:  (x- DAY_0).days)
totals['time of day'] = totals['time'].apply(lambda x: x.time())

#### Select relevant columns and save:

In [24]:
totals = totals[['interventionId', 'encounterId', 'minutes since admission', 'record day', 'time of day' ,'careProviderId', 'hourTotal', 'cumTotal', 'unitOfMeasure', 'longLabel']]
totals.to_csv('anonymised_total_balances.csv')
print "There are %d ICU stays in this data frame." %len(totals.encounterId.unique())

There are 5323 ICU stays in this data frame.


Note: there are fewer patients in the fluid balance data frame than in the patient summary.

### 4. Then process PtLabResult data: 

In [25]:
labres = pd.read_csv('labresults_physiological_data.rpt', delimiter='\t')

#### Same workflow as above:

In [26]:
to_drop = [i for i,row in labres.iterrows() if row['encounterId'] not in anon['encounterId'].values]
labres = labres.drop(to_drop, axis=0)

In [27]:
labres['time'] = pd.to_datetime(labres['chartTime'], infer_datetime_format=True)
labres = labres.merge(anon, on='encounterId')
labres['minutes since admission'] = (labres['time'] - labres['inTime']) / pd.Timedelta(minutes=1)
labres['record day'] = labres['time'].apply(lambda x:  (x- DAY_0).days)
labres['time of day'] = labres['time'].apply(lambda x: x.time())

In [28]:
labres = labres[['interventionId', 'encounterId', 'minutes since admission', 'record day', 'time of day' ,'attribute', 'longLabel', 'valueNumber', 'valueString']]
labres.to_csv('anonymised_labres.csv')
print "There are %d ICU stays in this data frame." %len(labres.encounterId.unique())

There are 5399 ICU stays in this data frame.


### 5. Then process PtAssess data:

In [29]:
ptassess = pd.read_csv('ptassess_physiological_data.rpt', delimiter='\t')

In [30]:
to_drop = [i for i,row in ptassess.iterrows() if row['encounterId'] not in anon['encounterId'].values]
ptassess = ptassess.drop(to_drop, axis=0)

In [31]:
ptassess['time'] = pd.to_datetime(ptassess['chartTime'], infer_datetime_format=True)
ptassess = ptassess.merge(anon, on='encounterId')
ptassess['minutes since admission'] = (ptassess['time'] - ptassess['inTime']) / pd.Timedelta(minutes=1)
ptassess['record day'] = ptassess['time'].apply(lambda x:  (x- DAY_0).days)
ptassess['time of day'] = ptassess['time'].apply(lambda x: x.time())

In [32]:
ptassess = ptassess[['interventionId', 'encounterId', 'minutes since admission', 'record day', 'time of day' ,'attribute', 'longLabel', 'valueNumber', 'valueString']]
ptassess.to_csv('anonymised_ptassess.csv')
print "There are %d ICU stays in this data frame." %len(ptassess.encounterId.unique())

There are 5375 ICU stays in this data frame.


### 6. We now split all the data frames into separate files (one for each ICU stay), to facilitate partial analysis:

In [33]:
for ii in anon['encounterId']:
    
    subset1 = totals[totals['encounterId']==ii]
    subset2 = targets[targets['encounterId']==ii]
    subset3 = labres[labres['encounterId']==ii]
    subset4 = ptassess[ptassess['encounterId']==ii]
    
    subset1.to_csv('patient_files/total_balances_%d.csv' %ii)
    subset2.to_csv('patient_files/targets_%d.csv' %ii)
    subset3.to_csv('patient_files/labres_%d.csv' %ii)
    subset4.to_csv('patient_files/ptassess_%d.csv' %ii)

### 7. Having processed all the data we can remove 'inTime' from the patient summary data frame and save it:

In [34]:
anon = anon[['encounterId', 'age', 'lengthOfStay (mins)', 'gender', 'weight', 'height', 'admission month', 'admission day', 'admission time']]
anon.to_csv('anonymised_patient_summary.csv')